In [ ]:
%matplotlib inline


#essentials
import numpy as np
import pylab as plt
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
import healpy as hp
from datetime import datetime
from pygdsm import GlobalSkyModel
from pygdsm import GSMObserver


#non essential
import time
from tqdm import tqdm
from astropy.coordinates import Angle
import numpy.ma as ma

# Quick Plot

### haslam as basemap

In [ ]:
gsm = GlobalSkyModel(interpolation='cubic', basemap='haslam')

freq = 150 # MHz unit
gsm.generate(freq)  
gsm.view(logged=True)

### wmap as basemap

In [ ]:
gsm = GlobalSkyModel(freq_unit='MHz', interpolation='cubic', basemap='wmap')

freq = 150 # MHz unit
gsm.generate(freq)  
gsm.view(logged=True)

# MollView Array and Spectral Index

In [ ]:
gsm = GlobalSkyModel(freq_unit='MHz', interpolation='cubic', basemap='haslam')

### brightness temperature vs freq

In [ ]:
map_cube.shape

In [ ]:
freqs = np.linspace(10, 1000, 100)
map_cube = gsm.generate(freqs)

print(map_cube.shape)
plt.loglog(freqs, map_cube[:,0])
plt.loglog(freqs,map_cube[:,123])
plt.loglog(freqs,map_cube[:,1546])
plt.loglog(freqs,map_cube[:,166])
plt.xlabel("Frequency [MHz]")
plt.ylabel("Temperature [K]")

In [ ]:
map_cube.shape

## calculating spectral index

In [ ]:
freqs = np.array([45,150,408,1420]) #frequencies of map

map_cube = gsm.generate(freqs)

In [ ]:
spectral_index_1 = -1*np.log(map_cube[1]/map_cube[0])/np.log(freqs[1]/freqs[0])

In [ ]:
spectral_index_2 = -1*np.log(map_cube[3]/map_cube[2])/np.log(freqs[3]/freqs[2])

In [ ]:
hp.mollview(spectral_index_1,cmap='inferno',norm='log')

In [ ]:
hp.mollview(spectral_index_2,cmap='inferno',norm='log')

# Visible region of the sky

In [ ]:
(latitude, longitude, elevation) = ('20.2961', '85.8245', 45) #Bhubaneswar Odisa
ov = GSMObserver()
ov.lon = longitude
ov.lat = latitude
ov.elev = elevation
ov.date = datetime(2023, 6, 12, 17, 30)

In [ ]:
ov.generate(150)
sky_dome_view = ov.view(logged=True)

In [ ]:
sky_moll_view = ov.view_observed_gsm()

In [ ]:
sky_moll_view

In [ ]:
NSIDE=16

low_res_sky = hp.pixelfunc.ud_grade(sky_moll_view,nside_out=NSIDE)

In [ ]:
# moll full view
hp.mollview(map_cube[1],cmap='inferno',unit='K',norm='hist')

In [ ]:
hp.mollview(sky_moll_view,cmap='inferno',unit='K',norm='hist')

In [ ]:
hp.mollview(low_res_sky,cmap='inferno',unit='K',norm='hist')

In [ ]:
npix = len(sky_moll_view)

In [ ]:
print(f"Angular resolution of the original map: \
{np.degrees(hp.pixelfunc.nside2resol(hp.pixelfunc.npix2nside(npix))):.2f} degrees")
print(f"NSIDE of the original map: {hp.pixelfunc.npix2nside(npix)}")
print(f"Angular resolution: {np.degrees(hp.pixelfunc.nside2resol(NSIDE)):.2f} degrees")
print(f"Number of pixels: {hp.pixelfunc.nside2npix(NSIDE)}")

# Converting converting healpy_pxl_idx to local alt-az

In [ ]:
ipix = int(3145728/16)
nside = hp.npix2nside(3145728)
theta, phi = hp.pix2ang(nside, ipix)
ra = np.rad2deg(phi) # ra in degrees in degrees ICRS
dec = np.rad2deg(0.5 * np.pi - theta) # dec in degrees ICRS

print(ra)
print(dec)

In [ ]:
c_icrs = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
c_icrs.galactic 

In [ ]:
(latitude, longitude, elevation) = ('20.2961', '85.8245', 100)
time = datetime(2023, 12, 12, 17, 30)


earth_location = EarthLocation(lat=float(latitude)*u.deg, lon=float(longitude)*u.deg,height=elevation*u.m)
icrs_coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')

trans_local = icrs_coord.transform_to(AltAz(obstime=time, location=earth_location))

az, alt                     = trans_local.az.degree, trans_local.alt.degree

print(f'alt:{alt}')
print(f'az:{az}')

# Beam Weighting

## this is where your assignment part comes in

#### 1. Weight the visible sky in your region with a $\cos^{2}\theta$(where $\theta$ is alt angle) beam and take mean.

#### 2. Do it for multiple frequencies and find the beam. 

#### 3. plot the means against frequency axis and try modeling it.